In [87]:
import os
import sys
import pandas as pd

import evo_helpers as hlp
import evo_constants as CTS

In [ ]:
hlp.welcome()

In [ ]:
#from importlib import reload
#reload(evo_helpers)

opt = hlp.option_menu()

In [ ]:
class Evomining:
    """This class creates an evomining object.
    Its methods permit to access to the evomining
    funtionality.
    """

    def __init__(self, centralDB,
                evo_genomesDB,
                nat_prodsDB):
        self.central = centralDB
        self.genomesDB = evo_genomesDB
        self.nat_prods = nat_prodsDB

In [ ]:
## obtener las bases de datos del usuario

#from importlib import reload
#reload(evo_helpers)

## option 1 precharged example:
if opt == 1:
    hlp.make_all_evo_headers(CTS.EXMPL_DB["genomes"],
                                 CTS.EXMPL_DB["rast_ids"])
    hlp.join_evo_headers_files()
    hlp.remove_evo_headers_files()
    
    evo_genomes_path = CTS.EVO_GENOMES_DB
    
    evomining = EvoMining(cts.EXML_DB["central"],
                          cts.EXMPL_DB["genomes"],
                          cts.EXMPL_DB["nat_prods"])
    #evomining.run()
## option 2 user databases:
else: 
    user_dbs = hlp.get_dbs()
    hlp.make_all_evo_headers(user_dbs["genomes"], 
                             user_dbs["rast_ids"])
    hlp.join_evo_headers_files()
    hlp.remove_evo_headers_files()
    
    user_dbs["genomes"] = CTS.EVO_GENOMES_DB
    
    evomining = EvoMining(user_dbs["central"],
                         user_dbs["genomes"],
                         user_dbs["nat_prods"])
    #evomining.run()

#evomining.run()

## Obtener los archivos .faa con los encabezados en formato evomining:
### Los nuevos .faa individuales se guardan en data/aux_data/  (se borran una vez utilizados)
### La base de datos con encabezados de evomining se guarda en /data/data_bases/evo_genomes_db/

In [ ]:
#from importlib import reload
#reload(evo_helpers)

genomes_db = "/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/example/genomes/los17"
Rast_ids = "/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/example/genomes/los17Rast.ids"
hlp.make_all_evo_headers(genomes_db, Rast_ids)
hlp.join_evo_headers_files()
hlp.remove_evo_headers_files()

## Aplicar makeblastdb a las bases de datos que necesitaremos:

In [ ]:
hlp.makeblast_db(CTS.EVO_GENOMES_DB)

In [ ]:
hlp.makeblast_db(CTS.EXMPL_DB["central"])


In [ ]:
hlp.makeblast_db(CTS.EXMPL_DB["nat_prods"])

## Obtener los blastp entre la base de datos central vs la base de datos de genomas:

In [ ]:
blastdb_path = CTS.BLASTDBs_PATH + 'evo_genomes_db_blastdb/'
            
hlp.apply_blastp(CTS.EXMPL_DB["central"], blastdb_path)

In [ ]:
CTS.EVO_GENOMES_DB

## Obtener los blastp entre la base de datos de genomas vs la base de datos central:

In [ ]:
blastdb_path = CTS.BLASTDBs_PATH + 'central_blastdb/'
            
hlp.apply_blastp(CTS.EVO_GENOMES_DB, blastdb_path)

## Obtener los BBH:

### Obtener los mejores elementos únicos del blast de metabolismo central vs genomas: 
### Estos se guardarán en un diccionario de la forma:
### { enzima_i : mejor_copia_en_el_organismo_k }
### donde i recorre todas las enzimas en el metabolismo central, y k recorre todos los genomas.
### - Considerar solo con bitscore >= 100 e E_value < 0.001 

In [24]:
df1 = pd.read_csv('/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/blastp/central_to_genomes/central_to_genomes.blast', sep = "\t", names = CTS.BLAST_COLS)

In [ ]:
central_enzimes = df1['query'].unique() ##Lista con las enzimas del metabolismo central
genomes = hlp.dict_ids_to_names_genomes(CTS.EXMPL_DB["rast_ids"]) ## Diccio con los genomas de la base de genomas

In [ ]:
df1.head()

In [ ]:
#definimos el diccionario que será la salida:
unique_in_first_blast = dict()
#inicializamos el diccionario tomando por claves todas las enzimas
# y por el momento asignando por valores listas vacías:
for enzime in central_enzimes:
    unique_in_first_blast[enzime] = list()
    
#definimos una lista auxiliar que nos permitirá quedarnos con la mejor
# copia de la enzima presente en cada organismo 

aux_list = list()
for enzime in central_enzimes:
    for genome_id in genomes.keys():
        for index in range(len(df1)):
            if (genome_id in df1["subject"][index]) & (aux_list == []):
                aux_list=list(df1.iloc[index])
            elif ((genome_id in df1["subject"][index]) & (df1["bitscore"][index] > aux_list[11])):
                aux_list = list(df1.iloc[index])
            else:
                continue
    unique_in_first_blast[enzime].append(aux_list)
    

In [ ]:
central_enzimes

In [ ]:
genomes.keys()

In [ ]:
#definimos el diccionario que será la salida:
unique_in_first_blast = dict()
#inicializamos el diccionario tomando por claves todas las enzimas
# y por el momento asignando por valores listas vacías:
for enzime in central_enzimes:
    unique_in_first_blast[enzime] = []
    
#definimos una lista auxiliar que nos permitirá quedarnos con la mejor
# copia de la enzima presente en cada organismo 

#aux_list = list()
for enzime in central_enzimes:
    for genome_id in genomes.keys():
        aux_list = []
        if genome_id != "6666666.385189":
            for index in range(len(df1)):
                if (aux_list == []):
                    if ((enzime == df1["query"][index])&(genome_id in df1["subject"][index])):
                        aux_list = list(df1.iloc[index])
                    else:
                        continue
                else:
                    if ((enzime == df1["query"][index]) & (genome_id in df1["subject"][index]) & (df1["bitscore"][index] > aux_list[11])):
                        aux_list = list(df1.iloc[index])
            if aux_list != []:  
                unique_in_first_blast[enzime].append(aux_list)
                


In [ ]:
unique_in_first_blast["3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none"]

In [ ]:
total = 0
for enz in central_enzimes:
        total = total + len(unique_in_first_blast[enz])
total

### NOTA: En la base de datos de genomas está de más el genoma con id = 6666666.385189

Unique de Genomes vs EF:

In [74]:
df2 = pd.read_csv('/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/blastp/genomes_to_central/genomes_to_central.blast', sep = "\t", names = CTS.BLAST_COLS )

In [79]:
df2.head()

,query,subject,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
0,gi|649831.6.513|649831.6|NC_021191.1|Phosphose...,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,62.810,363,135,0,11,373,1,363,4.600000e-174,479.0
1,gi|649831.6.513|649831.6|NC_021191.1|Phosphose...,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,57.995,369,154,1,5,373,9,376,1.970000e-156,435.0
2,gi|649831.6.513|649831.6|NC_021191.1|Phosphose...,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,53.639,371,171,1,1,371,1,370,7.230000e-144,403.0
3,gi|649831.6.824|649831.6|NC_021191.1|hypotheti...,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,27.711,83,53,2,14,96,164,239,1.880000e-05,31.6
4,gi|649831.6.1388|649831.6|NC_021191.1|D-3-phos...,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,36.491,285,159,5,42,314,39,313,4.960000e-49,164.0


In [ ]:
#definimos el diccionario que será la salida:
unique_in_second_blast = dict()
#inicializamos el diccionario tomando por claves todas las enzimas
# y por el momento asignando por valores listas vacías:
for genome_id in genomes.keys():
    if genome_id != "6666666.385189":
        unique_in_second_blast[genome_id] = []
    else:
        continue

#definimos una lista auxiliar que nos permitirá quedarnos con la mejor
# copia de la enzima presente en cada organismo 


for genome_id in genomes.keys():
    if genome_id != "6666666.385189":
        for enzime in central_enzimes:
            aux_list = []
            for index in range(len(df2)):
                if (aux_list == []):
                    if ((genome_id in df2["query"][index])&(enzime in df2["subject"][index])):
                        aux_list = list(df2.iloc[index])
                    else:
                        continue
                else:
                    if ((genome_id in df2["query"][index]) & (enzime in df2["subject"][index]) & (df2["bitscore"][index] > aux_list[11])):
                    
                        aux_list = list(df2.iloc[index])
            if aux_list != []:  
                unique_in_second_blast[genome_id].append(aux_list)


### Prueba de bio.genbank package:

In [ ]:
from taxonomy_ranks import TaxonomyRanks

In [ ]:
taxa_name = 'Actinomadura hibisca'

rank_taxon = TaxonomyRanks(taxa_name)

In [ ]:
type(rank_taxon)

In [ ]:
rank_taxon.get_lineage_taxids_and_taxanames()


In [ ]:
ranks = ('user_taxa', 'taxa_searched', 'superkingdom', 'kingdom', 'superphylum', 'phylum', 'subphylum', 'superclass', 'class', 'subclass', 'superorder', 'order', 'suborder', 'superfamily', 'family', 'subfamily', 'genus', 'subgenus', 'species')

In [ ]:
for potential_taxid in rank_taxon.lineages:
     for rank in ranks:
         if rank in ('user_taxa', 'taxa_searched'):
             taxon = rank_taxon.lineages[potential_taxid][rank]
             print(potential_taxid, rank, taxon, sep='\t')
         else:
             taxon, taxid_of_taxon = rank_taxon.lineages[potential_taxid][rank]
             print(potential_taxid, rank, taxon, taxid_of_taxon, sep='\t')


## Probar helpers para BBH:
**<center>1.- best_hit_central_to_orgs:</center>**

In [1]:
import evo_helpers as hlp
import evo_constants as CTS

In [11]:
blast_File = '/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/blastp/central_to_genomes/central_to_genomes.blast'
rast_Ids = CTS.EXMPL_DB["rast_ids"]

In [ ]:
BH_forw = hlp.best_hit_central_to_orgs(blast_File, rast_Ids)

In [ ]:
total = 0
for key in BH_forw:
    total += len(BH_forw[key])
total

### best_hit_central_to_genomes está bien!

In [4]:
def make_central_file_info():
    """This function creates a file with the names of all
    the elements in central_db.
    """
    central_fasta_file_path = CTS.EXMPL_DB["central"] + "central.fasta"
    output_path = CTS.BBH_aux_files + "central_enzimes.txt"
    with open(central_fasta_file_path,"r") as fasta_file:
        with open(output_path, "w") as output:
            for line in fasta_file:
                if ">" in line:
                    Line = line.split(">")[1]
                    output.write(Line)
    return

make_central_file_info()

In [8]:
def ids_to_names_genomes_file(file_rast_ids):
    """ This function returns a file with the genome id's as keys
    and the genome's names as values. The input file_rast_ids is the Rast.ids
    file within the genomes' database.
    """
    
    
    output_path = CTS.BBH_aux_files + "ids_to_orgs_names.txt"
    with open(file_rast_ids, "r") as file:
        with open(output_path, "w") as output:
            for line in file:
                Line = line.split("\t")[1] + "-->" + line.split("\t")[2]
                output.write(Line)
            
    return 
rast_ids_File = CTS.EXMPL_DB["rast_ids"]
ids_to_names_genomes_file(rast_ids_File)

# Por hacer:
- Agregar una parte en la que se creen todos los archivos auxiliares al comienzo de la rutina, para que estén disponibles en las subrutinas.

## Prueba best hit 1 con archivos:

In [15]:
def best_hit_central_to_orgs(blast_file):
    """ This function will obtain the best hit from central to organisms. Its output 
    is a file with these hits. From the output we will obtain the expanded families.
    """
    
    # Charge the dataframe with the blastp file:
    df = pd.read_csv(blast_file, sep = '\t', names = CTS.BLAST_COLS)
    
    #Obtain the different central enzimes:
    #central_enzimes = df['query'].unique()
    
    #Files with the enzimes and orgs info:
    central_enzimes_file = CTS.BBH_aux_files + 'central_enzimes.txt'
    central_enzimes = []
    with open(central_enzimes_file, "r") as central_file:
        for line in central_file:
            if line:
                central_enzimes.append(line.rstrip("\n"))
    
    ids_to_orgs_names_file = CTS.BBH_aux_files + 'ids_to_orgs_names.txt'
    genomes = dict()
    with open(ids_to_orgs_names_file, "r") as orgs_file:
        for line in orgs_file:
            if line:
                key = line.split("-->")[0]
                value = line.split("-->")[1]
                genomes[key] = value
    
    #Obtain a dictionary of the form {genome_id: "organism_name"}
    #genomes = dict_ids_to_names_genomes(rast_ids)
    
    #Define and initialize the output dictionary:
    unique_central_to_genomes = dict()
    for enzime in central_enzimes:
        unique_central_to_genomes[enzime] = []
        
    # Finding the best hit:
    for enzime in central_enzimes:
        for genome_id in genomes.keys():
            aux_list = []
            if genome_id != "6666666.385189":
                for index in range(len(df)):
                    if (aux_list == []):
                        if ((enzime == df["query"][index])&(genome_id in df["subject"][index])):
                            aux_list = list(df.iloc[index])
                        else:
                            continue
                    else:
                        if ((enzime == df["query"][index]) & (genome_id in df["subject"][index]) & (df["bitscore"][index] > aux_list[11])):
                            aux_list = list(df.iloc[index])
                if aux_list != []:  
                    unique_central_to_genomes[enzime].append(aux_list)
     
    return unique_central_to_genomes

In [20]:
test = best_hit_central_to_orgs(blast_File)
tot = 0
for enz in test:
    tot += len(test[enz])
tot

119

In [17]:
def best_hit_central_to_orgs_2(blast_file):
    """ This function will obtain the best hit from central to organisms. Its output 
    is a file with these hits. From the output we will obtain the expanded families.
    """
    
    # Charge the dataframe with the blastp file:
    df = pd.read_csv(blast_file, sep = '\t', names = CTS.BLAST_COLS)
    
    ### SE NECESITA QUE LOS ARCHIVOS AUXILIARES ESTÉN CREADOS ###
    #Files with the enzimes and orgs info:
    
    # Make a list with the enzimes:
    central_enzimes_file = CTS.BBH_aux_files + 'central_enzimes.txt'
    central_enzimes = []
    with open(central_enzimes_file, "r") as central_file:
        for line in central_file:
            if line:
                central_enzimes.append(line.rstrip("\n"))
    
    
    # Make a dictionary with the ids and names of organisms:
    ids_to_orgs_names_file = CTS.BBH_aux_files + 'ids_to_orgs_names.txt'
    genomes = dict()
    with open(ids_to_orgs_names_file, "r") as orgs_file:
        for line in orgs_file:
            if line:
                key = line.split("-->")[0]
                value = line.split("-->")[1]
                genomes[key] = value
    
    
    
    #Define and initialize the output dictionary:
    unique_central_to_genomes = dict()
    for enzime in central_enzimes:
        unique_central_to_genomes[enzime] = []
        
    # Finding the best hit:
    for enzime in central_enzimes:
        for genome_id in genomes.keys():
            aux_list = []
            
            for index in range(len(df)):
                if (aux_list == []):
                    if ((enzime == df["query"][index])&(genome_id in df["subject"][index])):
                        aux_list = list(df.iloc[index])
                    else:
                        continue
                else:
                    if ((enzime == df["query"][index]) & (genome_id in df["subject"][index]) & (df["bitscore"][index] > aux_list[11])):
                        aux_list = list(df.iloc[index])
            if aux_list != []:  
                unique_central_to_genomes[enzime].append(aux_list)
     
    return unique_central_to_genomes

In [18]:
best_hit_central_to_orgs_2(blast_File)

{'3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_1|Cglu': [['3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_1|Cglu',
   'gi|1156913.7.6305|1156913.7|NC_021252.1|D-3-phosphoglycerate_dehydrogenase_(EC_1.1.1.95)|Amycolatopsis_orientalis_HCCB10007_NC_021252.1',
   59.203,
   527,
   214,
   1,
   1,
   526,
   1,
   527,
   0.0,
   613.0],
  ['3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_1|Cglu',
   'gi|169133.3.2692|169133.3|LAB|D-3-phosphoglycerate_dehydrogenase_(EC_1.1.1.95)|Citricoccus_sp_CH26_LAB',
   46.183,
   524,
   280,
   2,
   1,
   523,
   1,
   523,
   2.16e-155,
   455.0],
  ['3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_1|Cglu',
   'gi|290340.24.2545|290340.24|NC_008711.1|D-3-phosphoglycerate_dehydrogenase_(EC_1.1.1.95)|Arthrobacter_aurescens_TC1_NC_008711.1',
   49.049,
   526,
   267,
   1,
   6,
   530,
   5,
   530,
   2.0600000000000003e-161,
   470.0],
  ['3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_1|Cglu',
   'gi|367928.21.1188|367928.21|NC_008618.

In [22]:
test2 = best_hit_central_to_orgs_2(blast_File)
tot1 = 0
for enz in test2:
    tot1 += len(test2[enz])
tot1

119

In [41]:
li = list(df1.iloc[0])
li = [str(el) for el in li]
"\t".join(li)

'3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_1|Cglu  gi|6666666.146852.1090|6666666.146852|NC_002935.2|D-3-phosphoglycerate_dehydrogenase_(EC_1.1.1.95)|Corynebacterium_diphtheriae_NCTC_13129_NC_002935.2  79.623  530  108  0  1  530  1  530  0.0  847.0'

In [114]:
def best_hit_central_to_orgs_3(blast_file):
    """ This function will obtain the best hit from central to organisms. Its output 
    is a file with these hits. From the output we will obtain the expanded families.
    """
    
    # Charge the dataframe with the blastp file:
    df = pd.read_csv(blast_file, sep = '\t', names = CTS.BLAST_COLS)
    
    ### SE NECESITA QUE LOS ARCHIVOS AUXILIARES ESTÉN CREADOS ###
    #Files with the enzimes and orgs info:
    
    # Make a list with the enzimes:
    central_enzimes_file = CTS.BBH_aux_files + 'central_enzimes.txt'
    central_enzimes = []
    with open(central_enzimes_file, "r") as central_file:
        for line in central_file:
            if line:
                central_enzimes.append(line.rstrip("\n"))
    
    
    # Make a dictionary with the ids and names of organisms:
    ids_to_orgs_names_file = CTS.BBH_aux_files + 'ids_to_orgs_names.txt'
    genomes = dict()
    with open(ids_to_orgs_names_file, "r") as orgs_file:
        for line in orgs_file:
            if line:
                key = line.split("-->")[0]
                value = line.split("-->")[1]
                genomes[key] = value
    
    
    
    #Define and initialize the output dictionary:
    #unique_central_to_genomes = dict()
    #for enzime in central_enzimes:
    #    unique_central_to_genomes[enzime] = []
     
    # Path to best hits file:
    
    central_to_orgs_bh = CTS.BBH_aux_files + "central_to_orgs_BH.uniq"
    
    
    # Finding the best hit:
    best_hits_file = open(central_to_orgs_bh, "w")
    for enzime in central_enzimes:
        for genome_id in genomes.keys():
            aux_list = []
            
            for index in range(len(df)):
                if (aux_list == []):
                    if ((enzime == df["query"][index])&(genome_id in df["subject"][index])):
                        aux_list = list(df.iloc[index])
                    else:
                        continue
                else:
                    if ((enzime == df["query"][index]) & (genome_id in df["subject"][index]) & (df["bitscore"][index] > aux_list[11])):
                        aux_list = list(df.iloc[index])
            if aux_list != []: 
                line = ["{}\t".format(item) for item in aux_list]
                best_hits_file.writelines(line)
                best_hits_file.write("\n")
                
    best_hits_file.close() 
    #return unique_central_to_genomes
    return

In [115]:
best_hit_central_to_orgs_3(blast_File)

In [59]:
cadena = "Hola\ta\ntodos."

In [60]:
cadena

'Hola\ta\ntodos.'

### Prueba de escribir best hits segundo blast en un archivo:

In [122]:
def best_hit_orgs_to_central(blast_file):
    """ This function will obtain the best hit from organisms to central, i.e., 
    assigns to each position id of a blast hit the corresponding enzime that best 
    hits in that particular place.
    """
    
    #charge dataframe:
    df = pd.read_csv(blast_file, sep = '\t', names = CTS.BLAST_COLS, index_col = False)
    
    #Obtain dictionary with the copy id position to the line of blast
    
    uniq2 = dict()
    for index in df.index:
        actual_line = df.iloc[index]
        copy_id = actual_line["query"].split("|")[1]  #id of the hit position
        if copy_id in uniq2:
            if actual_line["bitscore"] > uniq2[copy_id][11]:
                uniq2[copy_id] = list(actual_line)
        else:
            if actual_line["bitscore"] > 0:
                uniq2[copy_id] = list(actual_line)
    
    
    # Write the file with the best hits:
    
    orgs_to_central_bh = CTS.BBH_aux_files + "orgs_to_central_BH.uniq"  #Agregar ruta a constantes
    best_hits_file = open(orgs_to_central_bh, "w")
     
    for key in uniq2:
        line = ["{}\t".format(item) for item in uniq2[key]] 
        best_hits_file.writelines(line)
        best_hits_file.write("\n")
    
    
    best_hits_file.close()
    return


In [123]:
path_second_blast = '/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/blastp/genomes_to_central/genomes_to_central.blast'
best_hit_orgs_to_central(path_second_blast)

In [124]:
orgs_to_central_bh = CTS.BBH_aux_files + "orgs_to_central_BH.uniq"
DF1 = pd.read_csv(orgs_to_central_bh, sep = "\t", names = CTS.BLAST_COLS, index_col = False)

In [125]:
central_to_orgs_bh = CTS.BBH_aux_files + "central_to_orgs_BH.uniq"
DF2 = pd.read_csv(central_to_orgs_bh, sep = "\t", names = CTS.BLAST_COLS, index_col = False)

In [126]:
DF2.head()

,query,subject,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
0,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|1156913.7.6305|1156913.7|NC_021252.1|D-3-ph...,59.203,527,214,1,1,526,1,527,0.000000e+00,613.0
1,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|169133.3.2692|169133.3|LAB|D-3-phosphoglyce...,46.183,524,280,2,1,523,1,523,2.160000e-155,455.0
2,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|290340.24.2545|290340.24|NC_008711.1|D-3-ph...,49.049,526,267,1,6,530,5,530,2.060000e-161,470.0
3,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|367928.21.1188|367928.21|NC_008618.1|D-3-ph...,34.098,305,182,6,7,299,6,303,1.040000e-41,155.0
4,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|442563.11.829|442563.11|AD011|D-3-phosphogl...,36.749,283,171,3,15,292,13,292,3.030000e-46,167.0


In [181]:
## Optimizar el tiempo de ejecución (checar si se puede utilizar programación dinámica)

bbh_list = list()
bbh = dict()
BBH_file = CTS.BBH_aux_files + "BBH.bbh"  #Agregar ruta a constantes
bbh_file = open(BBH_file, "w")

for ind in DF1.index:
    copy_id = DF1.iloc[ind]["query"].split("|")[1]
    enzime = DF1.iloc[ind]["subject"]
    #print(copy_id + "-->" + enzime)
    for indxx in DF2.index:
            if copy_id not in DF2.iloc[indxx]["subject"]:
                continue
            elif enzime not in DF2.iloc[indxx]["query"]:
                continue
            else:# (copy_id in DF2.iloc[indxx]["subject"]) & (enzime in DF2.iloc[indxx]["query"]):
                hit = copy_id + " -> " + enzime
                bbh_list.append(hit)
                bbh[copy_id] = enzime
                #bbh_file.write(copy_id + " ------> ")
                #bbh_file.write(enzime +"\t")
                #bbh_file.write(DF1.iloc[ind]["query"]+"\n")
                
                
                #continue
bbh_file.close()

In [182]:
bbh_list

['649831.6.513 -> 3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none',
 '649831.6.6869 -> 3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_4|Scoe',
 '367928.21.407 -> 3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none',
 '367928.21.1188 -> 3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_4|Scoe',
 '452863.24.923 -> 3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none',
 '452863.24.2320 -> 3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_4|Scoe',
 '6666666.111158.7934 -> 3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_4|Scoe',
 '6666666.111158.8940 -> 3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none',
 '6666666.111572.1121 -> 3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_4|Scoe',
 '6666666.111572.2478 -> 3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none',
 '169133.3.2262 -> 3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none',
 '169133.3.2692 -> 3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_4|Scoe',
 '6666666.146852.774 -> 3PGA_AMINOACIDS|2|Phosphose

In [161]:
def bbh_detector(central_to_orgs_bhs, orgs_to_central_bhs):
    """ This function obtains the best bidirectional hits between
    the blast of central to organisms and organisms to central respectively.
    """

    # charge dataframes:
    
    df = pd.read_csv(central_to_orgs_bhs, sep = '\t', names = CTS.BLAST_COLS, index_col = False)
    
    df1 = pd.read_csv(orgs_to_central_bhs, sep = '\t', names = CTS.BLAST_COLS, index_col = False)
    
    # Write the file with the bbh:
    
    BBH_file = CTS.BBH_aux_files + "BBH.bbh"  #Agregar ruta a constantes
    bbh_file = open(BBH_file, "w")
     
    for indx in df1.index:
        copy_id = df1.iloc[indx]["query"].split("|")[1]
        enzime = df1.iloc[indx][1]
        for indxx in df.index:
            if copy_id not in df.iloc[indxx]["subject"]:
                continue
            elif enzime not in df.iloc[indxx]["query"]:
                continue
            else:
                bbh[copy_id] = enzime
                bbh_file.write(copy_id + " -> ")
                bbh_file.write(enzime +"\t")
                bbh_file.write(DF1.iloc[ind]["query"]+"\n")
           
    
    bbh_file.close()
    return

In [162]:
cent_to_org_bh = CTS.BBH_aux_files + "central_to_orgs_BH.uniq"
org_to_cent_bh = CTS.BBH_aux_files + "orgs_to_central_BH.uniq"

In [163]:
bbh_detector(cent_to_org_bh, org_to_cent_bh)

## Comparar los bbh contra los obtenidos en evomining:


In [183]:
BBH_evo = list()
BBH_evo_path = "/home/csar/Proyectos/Posdoc/ALL_curado.fasta_MiBIG_DB.faa_los17/blast/BBH/los17/aux_files/vueltalos17.blast.bbh"
bbh_evo = open(BBH_evo_path, 'r')
for line in bbh_evo:
    hit = line.split("\t")[0]
    BBH_evo.append(hit)


bbh_evo.close()
BBH_evo_set = set(BBH_evo)
bbh_set = set(bbh_list)

In [184]:
BBH_evo_set == bbh_set

True

In [185]:
BBH_evo_set
#bbh_set

{'1156913.7.6305 -> 3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_3|Mtub',
 '1156913.7.7342 -> 3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_2|none',
 '169133.3.2262 -> 3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none',
 '169133.3.2692 -> 3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_4|Scoe',
 '290340.24.1033 -> 3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none',
 '290340.24.2545 -> 3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_4|Scoe',
 '367928.21.1188 -> 3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_4|Scoe',
 '367928.21.407 -> 3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none',
 '442563.11.458 -> 3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_2|none',
 '442563.11.829 -> 3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_1|Cglu',
 '446462.15.520 -> 3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_2|none',
 '446462.15.6083 -> 3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_3|Mtub',
 '452863.24.2320 -> 3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_4